In [0]:
!wget "https://cdn.skillenza.com/files/5ee0a2e1-bf6d-4173-8244-e75fa2d7bbe2/training.5k.zip"

--2019-01-06 04:28:21--  https://cdn.skillenza.com/files/5ee0a2e1-bf6d-4173-8244-e75fa2d7bbe2/training.5k.zip
Resolving cdn.skillenza.com (cdn.skillenza.com)... 13.35.125.84, 13.35.125.94, 13.35.125.18, ...
Connecting to cdn.skillenza.com (cdn.skillenza.com)|13.35.125.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137569412 (131M) [application/zip]
Saving to: ‘training.5k.zip’

training.5k.zip     100%[===================>] 131.20M  19.2MB/s    in 6.5s    

2019-01-06 04:28:32 (20.1 MB/s) - ‘training.5k.zip’ saved [137569412/137569412]



In [0]:
!wget "https://cdn.skillenza.com/files/7da538a3-4db6-46ea-a4f8-87a21368e5f5/testing.40k.zip"


--2019-01-06 04:28:42--  https://cdn.skillenza.com/files/7da538a3-4db6-46ea-a4f8-87a21368e5f5/testing.40k.zip
Resolving cdn.skillenza.com (cdn.skillenza.com)... 13.35.125.84, 13.35.125.94, 13.35.125.18, ...
Connecting to cdn.skillenza.com (cdn.skillenza.com)|13.35.125.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1099209878 (1.0G) [application/zip]
Saving to: ‘testing.40k.zip’

testing.40k.zip     100%[===================>]   1.02G  17.0MB/s    in 48s     

2019-01-06 04:29:30 (21.7 MB/s) - ‘testing.40k.zip’ saved [1099209878/1099209878]



In [0]:
from zipfile import ZipFile

In [0]:
zf = ZipFile('training.5k.zip', 'r')
zf.extractall('training_dir')
zf.close()

In [0]:
zf2 = ZipFile('testing.40k.zip','r')
zf2.extractall('testing_dir')
zf2.close()

In [0]:
import pandas as pd


In [0]:
sol = pd.read_csv('training_dir/training/solution.csv')

In [0]:
import cv2
import glob

In [0]:
import os
names=[]
def load_images_from_folder(folder):
    images = []
    files={}
    for number,filename in enumerate(sorted(os.listdir(folder))):
      files[number]= filename
    for number , filename in sorted(files.items()):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img ,(64,64))
        names.append(filename)
        if img is not None:
            images.append(img)
    return images

In [0]:
image = load_images_from_folder('training_dir/training/training')

In [0]:
len(image)

5000

In [0]:
lit =[]
for i in range(5000):
  lit.extend(names[i].split('.'))

In [0]:
new_lit =[]
for i in range(10000):
  if(i%2==0):
    new_lit.append(int(lit[i]))

In [0]:
lit2 =[]
for i in range(5000):
  lit2.append(new_lit[i] -1 )

In [0]:
sol1 = sol.reindex(lit2)

In [0]:
y = sol1['category']

In [0]:
y = y-1

In [0]:
import numpy as np

In [0]:
import keras

Using TensorFlow backend.


In [0]:
df_list = []
for i in range(5000):
  df_list.append(image[i].flatten())

In [0]:
Dataset = pd.DataFrame(df_list)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [0]:
from keras.utils import to_categorical
from keras.layers import BatchNormalization

In [0]:
target = to_categorical(y)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=2)

In [0]:
model.fit(Dataset.values.reshape(-1, 64,64,1), target.reshape(-1,6), validation_split=0.2, epochs=30, callbacks=[es])

In [0]:
model.save_weights('first1.h5')

In [0]:
model.load_weights('first.h5')

In [0]:
Images = load_images_from_folder('testing_dir/testing/')

In [0]:
lit =[]
for i in range(5000,45000):
  lit.extend(names[i].split('.'))

In [0]:
new_lit =[]
for i in range(80000):
  if(i%2==0):
    new_lit.append(int(lit[i]))

In [0]:
df_list = []
for i in range(40000):
  df_list.append(Images[i].flatten())

In [0]:
Dataset_testing = pd.DataFrame(df_list)

In [0]:
from keras.models import load_model

In [0]:
my_model = load_model('Model.h5')

In [0]:
prediction = my_model.predict(Dataset_testing.values.reshape(-1, 64,64,1))

In [0]:
probability_true = prediction[:,1]

In [0]:
model1 = Sequential()
model1.add(Conv2D(32, (5, 5),
          padding='same',
          data_format='channels_last',
          input_shape=(64,32,1)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, (5, 5), padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))

model1.add(Dense(6))
model1.add(Activation('softmax'))


In [0]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [0]:
model1.fit(Dataset.values.reshape(-1, 64,64,1), target.reshape(-1,6), validation_split=0.2, epochs=30, callbacks=[es])

Train on 4000 samples, validate on 1000 samples
Epoch 1/30
4000/4000 [==============================] - 269s 67ms/step - loss: 1.9277 - acc: 0.1852 - val_loss: 1.7535 - val_acc: 0.2370
Epoch 2/30
4000/4000 [==============================] - 266s 66ms/step - loss: 1.6203 - acc: 0.2665 - val_loss: 1.6042 - val_acc: 0.2700
Epoch 3/30
4000/4000 [==============================] - 266s 66ms/step - loss: 1.4679 - acc: 0.3135 - val_loss: 1.4431 - val_acc: 0.2970
Epoch 4/30
4000/4000 [==============================] - 266s 67ms/step - loss: 1.4053 - acc: 0.3443 - val_loss: 1.4087 - val_acc: 0.3220
Epoch 5/30
4000/4000 [==============================] - 266s 67ms/step - loss: 1.3539 - acc: 0.3655 - val_loss: 1.5242 - val_acc: 0.2850
Epoch 6/30
4000/4000 [==============================] - 267s 67ms/step - loss: 1.3162 - acc: 0.3920 - val_loss: 1.3478 - val_acc: 0.3230
Epoch 7/30
4000/4000 [==============================] - 268s 67ms/step - loss: 1.2772 - acc: 0.4055 - val_loss: 1.4893 - val_acc: 

In [0]:
model1.save_weights('second.h5')

In [0]:
model1.save('Model2.h5')

In [0]:
my_model = load_model('Model2.h5')

In [0]:
prediction = my_model.predict(Dataset_testing.values.reshape(-1, 64,64,1))

In [0]:
labels = np.argmax(prediction[:,:], axis=1)
submission = pd.DataFrame(new_lit)
submission['category'] = labels + 1
submission.columns = ['id', 'category']
sub = submission.sort_values(by=['id'])
sub = sub.reset_index(drop=1)

In [0]:
sub.to_csv('submission2.csv', index=False)

From here onwards, the model for the next submission starts


In [0]:
my_model = load_model('Model5.h5')

In [0]:
my_model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [0]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 64, 64, 32)        832       
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 60, 32)        25632     
_________________________________________________________________
batch_normalization_11 (Batc (None, 60, 60, 32)        128       
_________________________________________________________________
activation_20 (Activation)   (None, 60, 60, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 32)        0         
__________

In [0]:
Dataset = Dataset*255.0

In [0]:
Dataset.head()

0     1     2     3     4     5     6         7     8     9     ...   \
0  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  1.000000   1.0   1.0  ...    
1  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  1.000000   1.0   1.0  ...    
2  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  1.000000   1.0   1.0  ...    
3  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  0.866667   1.0   1.0  ...    
4  0.996078   1.0   1.0   1.0   1.0   1.0   1.0  1.000000   1.0   1.0  ...    

   4086  4087  4088      4089  4090  4091  4092  4093  4094  4095  
0   1.0   1.0   1.0  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  
1   1.0   1.0   1.0  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  
2   1.0   1.0   1.0  0.952941   1.0   1.0   1.0   1.0   1.0   1.0  
3   1.0   1.0   1.0  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  
4   1.0   1.0   1.0  1.000000   1.0   1.0   1.0   1.0   1.0   1.0  

[5 rows x 4096 columns]

In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
check = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

In [0]:
my_model.fit(Dataset.values.reshape(-1, 64,64,1), target.reshape(-1,6), validation_split=0.2, epochs=30, callbacks=[es, check])

Train on 4000 samples, validate on 1000 samples
Epoch 1/30
4000/4000 [==============================] - 277s 69ms/step - loss: 0.6392 - acc: 0.7460 - val_loss: 1.5309 - val_acc: 0.4450
Epoch 2/30
4000/4000 [==============================] - 277s 69ms/step - loss: 0.5574 - acc: 0.7790 - val_loss: 1.5063 - val_acc: 0.4190
Epoch 3/30
4000/4000 [==============================] - 275s 69ms/step - loss: 0.5191 - acc: 0.7983 - val_loss: 1.2169 - val_acc: 0.4730
Epoch 4/30
4000/4000 [==============================] - 275s 69ms/step - loss: 0.4798 - acc: 0.8160 - val_loss: 2.1770 - val_acc: 0.4100
Epoch 5/30
4000/4000 [==============================] - 275s 69ms/step - loss: 0.4270 - acc: 0.8333 - val_loss: 1.6198 - val_acc: 0.4270


In [0]:
my_model.save('Model5.h5')

In [0]:
my_model1 = load_model('Model4.h5')

In [0]:
Dataset_testing = Dataset_testing *255

In [0]:
Dataset_testing.head()

0     1     2         3     4         5         6     7     8     9     \
0   1.0   1.0   1.0  1.000000   1.0  1.000000  1.000000   1.0   1.0   1.0   
1   1.0   1.0   1.0  0.796078   1.0  1.000000  1.000000   1.0   1.0   1.0   
2   1.0   1.0   1.0  1.000000   1.0  0.992157  0.847059   1.0   1.0   1.0   
3   1.0   1.0   1.0  1.000000   1.0  1.000000  1.000000   1.0   1.0   1.0   
4   1.0   1.0   1.0  1.000000   1.0  1.000000  1.000000   1.0   1.0   1.0   

   ...   4086  4087  4088  4089      4090      4091     4092  4093  4094  4095  
0  ...    1.0   1.0   1.0   1.0  0.921569  1.000000  1.00000   1.0   1.0   1.0  
1  ...    1.0   1.0   1.0   1.0  1.000000  1.000000  1.00000   1.0   1.0   1.0  
2  ...    1.0   1.0   1.0   1.0  1.000000  0.992157  0.74902   1.0   1.0   1.0  
3  ...    1.0   1.0   1.0   1.0  1.000000  1.000000  1.00000   1.0   1.0   1.0  
4  ...    1.0   1.0   1.0   1.0  1.000000  1.000000  1.00000   1.0   1.0   1.0  

[5 rows x 4096 columns]

In [0]:
prediction = my_model.predict(Dataset_testing.values.reshape(-1, 64,64,1))

In [0]:
labels = np.argmax(prediction[:,:], axis=1)
submission = pd.DataFrame(new_lit)
submission['category'] = labels + 1
submission.columns = ['id', 'category']
sub = submission.sort_values(by=['id'])
sub = sub.reset_index(drop=1)

NameError: ignored

In [0]:
sub.to_csv('submission5.csv', index=False)